# Introducción a LangChain

## Antes de comenzar

**Requisitos necesarios:**
1. Debe tener una cuenta en OPENAI
2. Añadir una variable de entorno con la KEY [Más información](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)
3. Instalar dependencias, por la cantidad de dependencias se recomienda ejecutar sobre un entorno virtual.

**Dependencias necesarias para este Notebook:**
1. pip install openai
2. pip install tiktoken
3. pip install langchain
4. pip install "langserve[all]"
5. pip install langchain-cli
6. pip install langchain docarray

Es posible que le de un error similar a este:
 "Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
     For further information visit ...."
Se soluciona con la siguiente instrucción: **pip3 install pydantic==1.10.9**


In [73]:
%pip install openai
%pip install tiktoken
%pip install langchain
%pip install langchain-cli
%pip install langchain docarray

#En caso de error
#%pip install pydantic==1.10.9


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [74]:
from typing import List

from langchain.llms import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import BaseOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.vectorstores import DocArrayInMemorySearch

## Sección 1 - Primeros pasos

### 1.1 Ejemplo sencillo de llamadas a modelos (GPT)
LangChain dispone de dos métodos para llamar a los modelos LLM como GPT: **Modo Texto** y **Modo Chat**

**Modo texto:**
Toma un texto de entrada (por ejemplo una pregunta) y devuelve una respuesta de tipo texto.

In [75]:


llm = OpenAI()
text = "Java es un lenguaje orientado a Objetos?"
result = llm.invoke(text)
print(type(result)) #el tipo devuelto es un string
print(result)

<class 'str'>


Sí, Java es un lenguaje de programación orientado a objetos. Esto significa que los programas en Java están estructurados alrededor de objetos, en lugar de acciones y lógica. Los objetos contienen datos en forma de variables y funciones en forma de métodos. Esto permite a los programadores crear código que sea reutilizable y mantenible.


**Modo Chat:**

#Modo chat: se le envía un mensaje con una lista de entradas con el formato [Rol - contenido] y devuelve una respuesta de tipo Mensaje.

En Langchain existen algunos roles predeterminandos:

- **HumanMessage:** Para un mensaje que viene de un humano/usuario.
- **AIMessage:** Para un mensaje aportado por la AI/assistant.
- **SystemMessage:** Para un mensaje del sistema
- **FunctionMessage / ToolMessage:**  Un mensaje que contiene la salida de una función, API, etc.

Tambien puede configurar un rol específico a través de la clase ChatMessage.

In [76]:
#Modo chat: se le envía un mensaje con una lista de entradas con [Rol - texto] y devuelve una respuesta de tipo Mensaje.
chat_model = ChatOpenAI()
IAText="Soy un experto en Java"
HumanText="¿Java es un lenguaje orientado a Objetos?"
messages = [AIMessage(content=IAText), 
            HumanMessage(content=HumanText)]

result = chat_model.invoke(messages)
print(type(result)) #el tipo devuelto es un IAMessage, que tiene un campo content que es un string
print(result)

<class 'langchain_core.messages.ai.AIMessage'>
content='Sí, Java es un lenguaje de programación orientado a objetos. Esto significa que está diseñado para modelar el mundo real mediante la creación de objetos que tienen propiedades y comportamientos. En Java, todo es un objeto, excepto los tipos primitivos como int, boolean, etc. La programación orientada a objetos en Java se centra en la encapsulación, la herencia y el polimorfismo para organizar y estructurar el código de manera eficiente y modular.'


### Sección 1.2 - Prompt templates
Normalmente  no solemos pasar mensajes(prompts) simples a los modelos LLMs. Lo más común es pasar contexto y/o instrucciones adicionales para optimizar el resultado. En causisticas donde se requieran lanzar prompts muy similares podemos usar plantillas, por ejemplo:

- **Ejemplo Prompt 1**: "como experto en Python, explicame el siguiente código:"messages = [AIMessage(content=IAText)"
- **Ejemplo Prompt 2**: "como experto en Java, explicame el siguiente código:"public ResponseEntity<String> getPetById(@PathVariable Long petId)"

Como podemos ver, este tipo de Prompt puede ser muy recurrente en un contexto de desarrollo software. Por ello, podemos construir una Template Prompt:

- **Prompt Template**: "como experto en {lenguaje}, explicame el siguiente código: {codigo}"

veamos un ejemplo:


In [77]:
#Modo chat: se le envía un mensaje con una lista de entradas con [Rol - texto] y devuelve una respuesta de tipo Mensaje.

chat_model = ChatOpenAI()
AssistantTemplate="eres un experto en {language}"
HumanTemplate="Como experto en {language}, explicame que hace el siguiente código: {codigo}"


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", AssistantTemplate),
    ("human", HumanTemplate),
])

#usamos format_messages para mapear los valores específicos en la plantilla
message = chat_prompt.format_messages(language='Python', codigo="messages = [AIMessage(content=IAText)]")
print("mensaje formateado:", message)

#invocamos el modelo con el mensaje final ya formateado y mapeado con los valores específicos: Python, la traza y el código
result = chat_model.invoke(message)

print("resultado:", result)
print("El tipo del resultado es:", type(result))

mensaje formateado: [SystemMessage(content='eres un experto en Python'), HumanMessage(content='Como experto en Python, explicame que hace el siguiente código: messages = [AIMessage(content=IAText)]')]
resultado: content='El código que has proporcionado crea una lista llamada "messages" que contiene un objeto AIMessage. El objeto AIMessage parece ser una entidad de mensaje de una API o biblioteca externa.\n\nDentro del objeto AIMessage, hay un atributo llamado "content" que se establece en "IAText". Esto sugiere que "IAText" es una variable o valor que contiene el contenido del mensaje.\n\nEn resumen, el código crea una lista que contiene un único objeto AIMessage con un atributo "content" establecido en "IAText".'
El tipo del resultado es: <class 'langchain_core.messages.ai.AIMessage'>


Como podemos ver, **el resultado no es un String**, es un tipo message de Langchain(AIMessage) con el formato: conten: respuesta. ¿y si quisera convertir la salida a string?

### Sección 1.3 - Output parsers

Otras de las capacidades de LangChain es la facilidad con la que podemos adatpar la salida el tipo de dato que necesitamos. Haciendo uso de la la clase BaseOutputParser, podemos usar parseadores predefinidos o crear uno propio.

En el siguiente ejemplo, vamos a parsear el resultado anterior con el parse predefinido StrOutputParser() para convertirlo en un String:

In [78]:
parserResult = StrOutputParser().invoke(result)
print("El tipo del resultado es:", type(parserResult))
print("El resultado es:", parserResult)

El tipo del resultado es: <class 'str'>
El resultado es: El código que has proporcionado crea una lista llamada "messages" que contiene un objeto AIMessage. El objeto AIMessage parece ser una entidad de mensaje de una API o biblioteca externa.

Dentro del objeto AIMessage, hay un atributo llamado "content" que se establece en "IAText". Esto sugiere que "IAText" es una variable o valor que contiene el contenido del mensaje.

En resumen, el código crea una lista que contiene un único objeto AIMessage con un atributo "content" establecido en "IAText".


En el siguiente ejemplo crearemos un parseador de salida específico para convertir un string a una lista de strings separada por ",". 

In [79]:
class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")
    
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

"""Eres un asistente útil que genera listas separadas por comas.
Un usuario pasará una categoría y deberías generar 5 objetos en esa categoría en una lista separada por comas.
SOLO devuelve una lista separada por comas, y nada más."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "paises"})

['Argentina', 'Brasil', 'Colombia', 'México', 'Perú']

En este caso el resutado no es ni un mensaje ni un string. Es una lista de strings con el siguiente formato: *['Mexico', 'Canada', 'Brazil', 'Australia', 'France']*. Esto es porque hemos usado el parseador CommaSeparatedListOutputParser.

En este caso, para parsear el resultado se ha usado la instrucción: *chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()* pero ¿qué hace esto? ¿como funciona el operador |? resolveremos esas dudas en la siguiente sección.

## Sección 2 - (LCEL) LangChain Expression Language

El lenguaje LCEL nos permite, por ejemplo, encadenar una serie acciones en una sola línea con el comando | como vimos en la instrucción:
 
 *chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()* 
 
 Usando LCEL se reduce significativamente el número de líneas de codigo necesarias para construir una funcionalidad. En este caso, al cuando usamos el método invoke se pasa el valor de la variable text (paises) a la Prompt Template, despues se ejecuta el modelo a partir de la Prompt generada por la template, y por último se realiza el parseo de la salida del modelo a una lista separada por comas. Es decir, se ejecutan de izquierda a derecha todas las acciones dentro del Pipeline:

*chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()*

*chain.invoke({"text": "paises"})*

Veamos un ejemplo paso a paso:




In [80]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Cuentame una historia corta sobre {topic}")
chain = prompt | model
chain.invoke({"topic": "un perro"})

AIMessage(content='Había una vez un perro llamado Max, que vivía en un pequeño pueblo junto a su dueño, Juan. Max era un perro muy juguetón y le encantaba explorar cada rincón del pueblo.\n\nUn día, mientras Max paseaba por el parque, escuchó un ruido extraño proveniente de un arbusto. Curioso, se acercó corriendo y encontró a un cachorro abandonado. El pequeño perro estaba asustado y temblaba de frío.\n\nMax, con su corazón bondadoso, decidió cuidar del cachorro y llevarlo a su hogar. Juan se sorprendió al ver al nuevo integrante de la familia, pero al ver la tristeza en los ojos del cachorro, decidió que también merecía un hogar amoroso.\n\nCon el paso del tiempo, el cachorro fue creciendo y se convirtió en un perro leal y cariñoso. Max y su nuevo amigo, al que llamaron Bruno, se volvieron inseparables. Juntos, exploraban el pueblo y jugaban sin cesar.\n\nUn día, mientras Max y Bruno jugaban cerca de un lago, escucharon los gritos de una niña que estaba en problemas. Sin pensarlo dos

Ahora formateemos el mensaje a un string, en vez de la salida actual *AIMessage*.

In [81]:
chain = prompt | model | StrOutputParser()
chain.invoke({"topic": "un perro"})

'Había una vez un perro llamado Max, quien vivía en un pequeño pueblo junto a su dueño, Santiago. Max era un perro muy leal y siempre estaba dispuesto a proteger y cuidar a su amo.\n\nUn día, Santiago decidió ir de excursión a las montañas cercanas. Max estaba emocionado y no podía esperar para aventurarse en la naturaleza. Juntos, caminaron por senderos sinuosos, disfrutando del aire fresco y los hermosos paisajes.\n\nDe repente, mientras caminaban por un camino estrecho, Santiago tropezó y se cayó por un barranco. Max, lleno de preocupación, corrió rápidamente hacia él y ladró para llamar la atención de alguien que pudiera ayudar. Pero no había nadie cerca.\n\nSin perder tiempo, Max decidió bajar al barranco para ayudar a su dueño. Con cuidado, descendió por las rocas y raíces, hasta que finalmente llegó junto a Santiago. Afortunadamente, Santiago solo tenía algunas heridas leves, pero estaba atrapado y no podía moverse.\n\nMax, sin dudarlo, buscó una forma de llevar a su dueño de vu

Ahora, usaremos un nuevo comando Batch en vez de invoke. En este caso para ejecutar el pipeline para varias entradas:

In [82]:
chain.batch([{"topic": "Un perro"}, {"topic": "Un gato"}])

['Había una vez un perro llamado Max que vivía en un pequeño pueblo. Max era un perro muy curioso y aventurero, siempre buscando nuevas emociones y diversión. Un día, mientras paseaba por el bosque cercano, Max descubrió una misteriosa cueva.\n\nIntrigado, decidió entrar en la cueva y explorar su interior. Al principio, todo estaba oscuro y Max se movía con cautela, pero pronto comenzó a ver destellos de luz que provenían de lo profundo de la cueva. Siguiendo la luz, Max se adentró aún más y se encontró con un tesoro brillante y reluciente.\n\nEra un collar de diamantes que estaba tirado en el suelo. Max, emocionado, decidió tomar el collar y llevarlo a casa. Pero en el camino de regreso, se encontró con un niño llorando en el parque.\n\nEl niño le explicó a Max que había perdido a su perro y que estaba muy triste. Max, sintiendo empatía por el niño, decidió darle el collar de diamantes para animarlo. El niño, sorprendido y agradecido por el gesto de Max, le dio un abrazo y prometió cu

En esta ocasion, con el comando batch se ha ejecutado 2 veces el proceso. En el siguiente ejemplo usaremos stream, que nos permitirá imprimir el resultado palabra a palabra a medida que recibe la respuesta en vez de esperar a tener toda la respuesta.

In [83]:
for s in chain.stream({"topic": "un perro"}):
    print(s, end="", flush=True)

Había una vez un perro llamado Max, un pastor alemán muy juguetón y curioso. Vivía en un pequeño pueblo junto a su dueño, Juan, quien lo amaba con todo su corazón.

Un día, Juan encontró un mapa antiguo en el desván de su casa. Decidido a descubrir su misterio, decidió seguirlo junto a Max. El mapa los llevó a un bosque frondoso y desconocido, donde la aventura comenzó.

Mientras caminaban, Max percibió un olor peculiar y comenzó a excavar con entusiasmo. Descubrió un pequeño cofre enterrado. Juan lo abrió cuidadosamente y encontró un collar reluciente con una etiqueta que decía: "Para el perro más valiente".

Inmediatamente, Max reconoció que ese collar era para él. Se lo puso y, de repente, comenzó a hablar. Juan quedó sorprendido, pero Max le explicó que el collar tenía poderes mágicos que le permitían comunicarse con los humanos.

Con su nueva habilidad, Max y Juan continuaron adentrándose en el bosque, resolviendo enigmas y superando obstáculos. Juntos, se enfrentaron a un río cau

## Sección 3 - Jugando con Embeddings y Vectors DBs(In progress)

En el siguiente ejemplo se muestra un ejemplo de RAG sensillo donde se almacenan los embeddings en memoria.

In [87]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Proceso 1: Los empleados deben imputar sus horas en cada una de las tareas en las que ha trabajado cada día", 
     "Proceso 2: En caso de que el empleado no haya trabajado en una tarea, debe imputar las horas en la tarea 'Otros'",
     "Proceso 3: Las vacaciones se imputarán en la tarea 'Vacaciones', pero debe enviarse un mail a RRHH para que se contabilicen correctamente",
     "Proceso 4: Las horas extras se imputarán en la tarea 'Horas extras', pero debe enviarse un mail a RRHH para que se contabilicen correctamente",
     "Proceso 5: Las horas de formación se imputarán en la tarea 'Formación', el empleado debe firmar un documento de asistencia y enviarlo a RRHH",
     "Norma 1: El empleado debe imputar sus horas en el sistema antes de las 18:00 del día siguiente",
     "Norma 2: Las vacaciones deben tomarse para un perido mínimo de 5 días naturales"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Responde a la pregunta teniendo en cuenta en primer lugar el siguiente contexto sin ser muy creativo:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

print(chain.invoke("Hoy he asistido a una formación de 2H, y el resto del tiempo he trabajado de tareas. ¿cual es el procedimiento?"))

El procedimiento sería imputar las 2 horas de formación en la tarea "Formación" y el resto del tiempo trabajado en las tareas correspondientes. Además, es importante firmar un documento de asistencia y enviarlo a RRHH para que se registre la formación correctamente.
